In [30]:
%load_ext autoreload
%autoreload 2
import hydra
import sys
sys.path.append('../meshgraphnet')
from omegaconf import DictConfig, OmegaConf
from data.datamodule import MeshDataModule
from torch_geometric.loader import DataLoader
from tasks.CfdSequenceRegressionTask import CfdSequenceRegression
from tasks.ClothSeqenceRegressionTask import ClothSequenceRegression
from visualization.viz import generate_images
import torch
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
path = "/nfs/homedirs/yan/hiwi_project/meshgraphnet/multirun/2022-02-21/00-08-53/0/wandb/run-20220221_004611-fvtvtslv/files/checkpoints/best.ckpt"
model = CfdSequenceRegression.load_from_checkpoint(path, map_location='cuda')

In [31]:
cfg = {
    'dataset_name': 'cylinder_flow',
    'field': 'velocity',
    'history': False,
    'batch_size_train': 1,
    'batch_size_valid': 10,
    'data_dir': '/nfs/students/yan/data/cylinder_flow',
    'steps': [1, 10, 50, 100, 200],
    'max_step': 2
}
cfg = OmegaConf.create(cfg)

datamodule = MeshDataModule(cfg)
datamodule.prepare_data()
datamodule.setup()

valid_ds = datamodule.valid_ds
valid_loader = datamodule.val_dataloader()

prepare data ...
setup dataset ...
Number of data:  1000
Number of data:  100


In [32]:
trajectories = model.predict_trajectory(valid_loader, step=cfg.max_step)

 10%|█         | 1/10 [01:03<09:33, 63.73s/it]


KeyboardInterrupt: 

In [9]:
scalars = {'mse_%03d_steps' % step: 0.0 for step in cfg.steps}

num_data = len(trajectories)
for i in range(num_data):
    error = np.mean((valid_ds[i].target_velocity[:, :cfg.max_step].numpy() - trajectories[i].numpy())**2, axis=-1)
    scalars = {'mse_%03d_steps' % step:
               scalars['mse_%03d_steps' % step] + np.mean(error[:, :step])
               for step in cfg.steps}
scalars = {key: value / num_data for key, value in scalars.items()}
print(scalars)

{'mse_001_steps': 1.5965328702804982e-05, 'mse_010_steps': 7.930151750770164e-05, 'mse_050_steps': 0.000327620831594686, 'mse_100_steps': 0.0006434990214620484, 'mse_200_steps': 0.0012231732212239876}


In [34]:
path = '/nfs/homedirs/yan/outputs/2022-03-06/16-31-48/wandb/run-20220306_163149-39g5ua4g/files/checkpoints/best.ckpt'
model = ClothSequenceRegression.load_from_checkpoint(path, map_location='cuda')

In [38]:
cfg = {
    'dataset_name': 'flag_simple',
    'field': 'world_pos',
    'history': True,
    'batch_size_train': 1,
    'batch_size_valid': 20,
    'data_dir': '/nfs/students/yan/data/flag_simple',
    'steps': [1, 10, 50],
    'max_step': 50
}
cfg = OmegaConf.create(cfg)

datamodule = MeshDataModule(cfg)
datamodule.prepare_data()
datamodule.setup()

valid_ds = datamodule.valid_ds
valid_loader = datamodule.val_dataloader()

prepare data ...
setup dataset ...
Number of data:  1000
Number of data:  100


In [39]:
trajectories = model.predict_trajectory(valid_loader, step=cfg.max_step)

100%|██████████| 5/5 [2:01:32<00:00, 1458.41s/it]


In [41]:
scalars = {'mse_%03d_steps' % step: 0.0 for step in cfg.steps}

num_data = len(trajectories)
for i in range(num_data):
    error = np.mean((valid_ds[i].target_world_pos[:, :cfg.max_step].numpy() - trajectories[i].numpy())**2, axis=-1)
    scalars = {'mse_%03d_steps' % step:
               scalars['mse_%03d_steps' % step] + np.mean(error[:, :step])
               for step in cfg.steps}
scalars = {key: value / num_data for key, value in scalars.items()}
print(scalars)

{'mse_001_steps': 2.9271307386125046e-06, 'mse_010_steps': 0.0038158015801309374, 'mse_050_steps': 0.20464056944008915}
